In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import csv 
import re


In [ ]:
class reviews:
    def __init__(self, df):
        self.movies = df
        self.url = "https://www.imdb.com/"
        self.tag_words = []
        self.reviews = {}
        
        
    def make_tagword(self):
        pattern = r'^(.*?)\(.*[^)](.{5})$'
        self.unique_movies = list(self.movies['movie_title'].unique())
        for tag in self.unique_movies:
            match = re.search(pattern, tag)
            if match:
                before_bracket = match.group(1)
                last_five = match.group(2)
                self.tag_words.append(before_bracket + " " + last_five)
            else:
                self.tag_words.append(tag)


    def initiate_driver(self):
        service = Service('chromedriver.exe')    
        options = Options()
        options.headless = True
        self.driver = webdriver.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, 10)

        self.driver.get(self.url)
    
    def get_search_box(self):
        self.search_box = self.driver.find_element(By.ID, "suggestion-search")

    def make_search(self, tag):
        print(tag)
        self.search_box.send_keys(tag)
        self.search_box.send_keys(Keys.RETURN)
        link = self.driver.find_element(By.CLASS_NAME, 'ipc-metadata-list-summary-item__t')
        self.driver.execute_script("arguments[0].scrollIntoView();", link)
        self.driver.execute_script("arguments[0].click();", link)

    def click_reviews(self):
        user_review_links = self.driver.find_element(By.XPATH , '/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[7]/div[1]/div/a')
        self.driver.execute_script("arguments[0].scrollIntoView();", user_review_links)
        self.driver.execute_script("arguments[0].click();", user_review_links)
    
    def get_reviews(self, tag):
        div_elements = WebDriverWait(self.driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.text.show-more__control.clickable'))
        )
        for div in div_elements[:10]:
           self.reviews[tag] = div.text

    def go_back(self):
        self.driver.back()

    
        
rev = reviews(movie.df)

In [ ]:

rev.make_tagword()
rev.initiate_driver()
for tag in rev.tag_words[0:416]:
    try:
        rev.get_search_box()
        rev.make_search(tag)
        rev.click_reviews()
        rev.get_reviews(tag)
    except:
        print("Exception")
    finally:
        continue

In [ ]:
with open('reviews.csv', 'w', newline='', encoding='utf-8') as csvfile: 
    writer = csv.writer(csvfile)
    writer.writerow(["Movie", "Review"])
    for movie, review in rev.reviews.items():
        writer.writerow([movie, review])